In [8]:
%matplotlib notebook
%pylab
import scipy
import numpy
from IPython.display import display, HTML
import statsmodels.api as sm
import statsmodels.formula.api as smf

Using matplotlib backend: nbAgg
Populating the interactive namespace from numpy and matplotlib


In [3]:
import pandas
activity = pandas.read_csv("../processed/activity_features_aggregate.txt", index_col=0, sep="\t")
ukbb = pandas.read_hdf("../processed/ukbb_data_table.h5")
mental_health = pandas.read_hdf("../processed/ukbb_mental_health.h5")
by_day = pandas.read_csv("../processed/activity_by_day.txt", index_col=0, sep="\t")

In [16]:
# Down-sample for speed and testing
import numpy
numpy.random.seed(0)
selected_IDs = numpy.random.choice(by_day.ID.unique(), size=10_000, replace=False)
data = by_day[by_day.ID.map(lambda x: x in selected_IDs)].copy()

In [54]:
data['sex'] = data.ID.map(ukbb.sex)
data['day'] = data.index
data['day_of_week'] = pandas.Categorical(pandas.to_datetime(data.index).dayofweek, categories=range(7))
data.sex.describe()

count      63728
unique         2
top       Female
freq       36129
Name: sex, dtype: object

In [66]:
# Mixed-effects model
# Day and individual are random effects
# Sex is non-random covariate

model = smf.mixedlm("main_sleep_onset ~ sex + day_of_week", groups=[["day", "ID"]], data=data, missing="drop")
fit = model.fit()

ValueError: operands could not be broadcast together with shapes (63728,) (1,2) 

In [56]:
fit.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
            Mixed Linear Model Regression Results
==============================================================
Model:            MixedLM Dependent Variable: main_sleep_onset
No. Observations: 60532   Method:             REML            
No. Groups:       911     Scale:              2.3254          
Min. group size:  1       Likelihood:         -111594.3765    
Max. group size:  152     Converged:          Yes             
Mean group size:  66.4                                        
--------------------------------------------------------------
                  Coef.  Std.Err.    z     P>|z| [0.025 0.975]
--------------------------------------------------------------
Intercept         23.286    0.020 1155.397 0.000 23.247 23.326
sex[T.Male]        0.063    0.013    5.046 0.000  0.039  0.088
day_of_week[T.1]   0.056    0.028    2.030 0.042  0.002  0.110
day_of_week[T.2]   0.077    0.028    2.796 0.005  0.023  0.131
day_of_week[T.3]   0.113    0.028    4.097 0.000  0.059  0.168
day_of_week[T.4]   0.240    0.028    8.651 0.000  0.185  0.294
day_of_week[T.5]   0.328    0.028   11.876 0.000  0.274  0.382
day_of_week[T.6]   0.016    0.028    0.578 0.563 -0.038  0.070
day Var            0.013    0.002                             
==============================================================

"""

In [65]:
smf.ols("num_wakings ~ sex + day_of_week", data=data.drop_duplicates("ID"), missing="drop").fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            num_wakings   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     10.72
Date:                Tue, 05 Nov 2019   Prob (F-statistic):           1.60e-13
Time:                        14:14:23   Log-Likelihood:                -18640.
No. Observations:                9540   AIC:                         3.730e+04
Df Residuals:                    9532   BIC:                         3.735e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            1.1588      0.044     26.547      0.000       1.073       1.244
sex[T.Male]          0.2528      0.035      7.153      0.000       0.183       0.322
day_of_week[T.1]     0.0897      0.105      0.853      0.394      -0.117       0.296
day_of_week[T.2]    -0.0112      0.060     -0.187      0.851      -0.128       0.106
day_of_week[T.3]    -0.0484      0.059     -0.815      0.415      -0.165       0.068
day_of_week[T.4]     0.1140      0.056      2.052      0.040       0.005       0.223
day_of_week[T.5]     0.1847      0.055      3.332      0.001       0.076       0.293
day_of_week[T.6]     0.1244      0.128      0.969      0.333      -0.127       0.376
==============================================================================
Omnibus:                     8900.467   Durbin-Watson:                   1.966
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           875006.413
Skew:                           4.175   Prob(JB):                         0.00
Kurtosis:                      49.169   Cond. No.                         9.37
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Int64Index([0, 1, 2, 5, 6, 0, 1, 2, 3, 4,
            ...
            1, 2, 3, 4, 1, 2, 3, 4, 5, 6],
           dtype='int64', length=63728)